## Bibliotecas

In [ ]:
# 1.Bibliotecas
import pandas as pd
import inflection
import sweetviz as sv
import numpy as np
import folium
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from collections import namedtuple
from folium.plugins import MarkerCluster


## Importação da base de dados

In [ ]:
# 2. Importando a base de dados
df = pd.read_csv('dataset/zomato.csv')
df1 = pd.read_csv('dataset/zomato.csv')

## Limpezas e Funções

#### Funções

In [ ]:
# 3. Funções e dicionários

# 3.1 Criação do nome das Cores
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

def color_name(color_code):
    return COLORS[color_code]

# 3.2 Preenchimento do nome dos países
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

# 3.3 Criação do Tipo de Categoria de Comida
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

# 3.4 Renomear as colunas do DataFrame    
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new

    return df

# 3.5 Total de tipos de culinárias disponíveis
def fnc_tot_tp_culinaria( df ):
    vListaCuisines = ( set( df['cuisines'] ) )
    vListaTotalCuisine = []
    
    for vCuisine in vListaCuisines:
        vTeste = vCuisine.split(',')
        # print(vTeste)
        for vPalavra in vTeste:
            vPalavra = vPalavra.strip()
            # print(vPalavra)
            if vPalavra in vListaTotalCuisine:
                None
            
            else:
                vListaTotalCuisine.append( vPalavra )

    df_aux = pd.DataFrame(vListaTotalCuisine)
    
    return df_aux.count()

# 3.6 Calculos por país
def fnc_calc_per_country( flag, tp_calc, df ):
    v01 = 0
    v02 = 0
    vList01 = []
    vList02 = []

    # Quantidade de cidades registradas por pais
    if flag == 1:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( df['country_code'] == i ), [ 'city' ] ]
            v01 = len( df_aux01[ 'city' ].unique() )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'num_cities' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )

    # Quantidade de restaurantes registrados por pais    
    elif flag == 2:
        for i in df['country_name'].unique():
            df_aux01 = df.loc[ ( df['country_name'] == i ), [ 'restaurant_id' ] ]
            v01 = len( df_aux01[ 'restaurant_id' ].unique() )

            vList01.append( i )
            vList02.append( v01 )

            data = { 'countries': vList01,
                     'num_restaurants' : vList02 }

            data = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':

            data.columns = [ 'Países', 'Quantidade de Restaurantes' ]
            fig = go.Figure()
            fig = ( px.bar(data,
                         text='Quantidade de Restaurantes',
                        x='Países', y='Quantidade de Restaurantes').update_traces( marker_color='#66b3ff' )
                                                                    .update_layout( plot_bgcolor='white' )
                                                                    .update_xaxes( mirror=True,
                                                                                    ticks='outside',
                                                                                    showline=True,
                                                                                    linecolor='black',
                                                                                    gridcolor='lightgrey' )
                                                                    .update_yaxes( mirror=True,
                                                                                    ticks='outside',
                                                                                    showline=True,
                                                                                    linecolor='black',
                                                                                    gridcolor='lightgrey')
                                                                    )
            return fig
        
        elif tp_calc == 'maior':
            return vMaiorPais

    # Quantidade de restaurantes com nivel de preço igual a 4 (gourmet)    
    elif flag == 3:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( ( df['country_code'] == i) & ( df[ 'price_range' ] == 4 ) ), [ 'restaurant_id' ] ]
            v01 = len( df_aux01[ 'restaurant_id' ].unique() )

            vList01.append( country_name( i ) )
            vList02.append( v01 )

            data = { 'countries': vList01,
                     'num_restaurants' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
    
    # Quantidade de culinarias unicas
    if flag == 4:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( df['country_code'] == i ), [ 'cuisines' ] ]
            v01 = len( df_aux01[ 'cuisines' ].unique() )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'num_cuisines' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
        
    # Soma de avaliações por país
    if flag == 5:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( df['country_code'] == i ), [ 'votes' ] ]
            v01 = df_aux01[ 'votes' ].sum()

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'qt_votes' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
        
    # Quantidade de restaurantes que fazem entrega
    if flag == 6:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( ( df['country_code'] == i) & ( df[ 'is_delivering_now' ] == 1 ) ), : ]
            v01 = len( df_aux01 )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'qt_delivery' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )
        
    # Quantidade de restaurantes que aceitam reservas
    if flag == 7:
        for i in df['country_code'].unique():
            df_aux01 = df.loc[ ( ( df['country_code'] == i) & ( df[ 'has_table_booking' ] == 1 ) ), : ]
            v01 = len( df_aux01 )

            vList01.append( country_name( i ) )
            vList02.append( v01 )
            data = { 'countries': vList01,
                     'qt_booking' : vList02 }

            df_return = pd.DataFrame( data )

            if v01 > v02:
                v02 = v01
                vMaiorPais = i

        if tp_calc == 'grafico':
            return df_return
        
        elif tp_calc == 'maior':
            return country_name( vMaiorPais )

### Limpezas

In [ ]:
# 1. Renomeando as colunas
df = rename_columns( df )

# 2. Removenco as linhas em branco
# cuisines
df.dropna( subset=[ 'cuisines' ], inplace=True )

# 3. Criando a coluna cuisine_category

# 4. Criando a coluna country_name
df[ 'country_name' ] = df[ 'country_code' ].apply( lambda x: country_name( x ) )

# 5. Categorizando os restaurantes para somente um tipo de culinária(cuisines)
df['cuisines'] = df['cuisines'].astype(str)
df['cuisines'] = df.loc[ :, 'cuisines' ].apply( lambda x: x.split( ',' )[0] )

# 6. Removendo Cuisines do tipo Mineira e Drinks Only
df = df.loc[( df['cuisines'] != 'Drinks Only') & ( df['cuisines'] != 'Mineira' ) & ( df['cuisines'] != 'NaN' ), : ]

# 7. Removendo linhas duplicadas
df = df.drop_duplicates()

## Respostas das perguntas de negócio

### Geral

##### 1. Quantos restaurantes únicos estão registrados?

In [ ]:
vRestUnique = df[ 'restaurant_id' ].nunique()
vRestUnique

##### 2. Quantos países únicos estão registrados?

In [ ]:
vCountryUnique = df[ 'country_code' ].nunique()
vCountryUnique

#### 3. Quantas cidades únicas estão registradas?

In [ ]:
vCityUnique = df[ 'city' ].nunique()
vCityUnique

#### 4. Qual o total de avaliações feitas?

In [ ]:
vCountVote = df[ 'votes' ].sum()
vCountVote = '{:_}'.format( df[ 'votes' ].sum() ).replace( '.', '.' ).replace( '_', '.' )
vCountVote 

#### 5. Qual o total de tipos de culinária registrados?

In [ ]:
vTotalCuisines = fnc_tot_tp_culinaria( df )
vTotalCuisines

### País

##### 1. Qual o nome do país que possui mais cidades registradas?

In [ ]:
print( fnc_calc_per_country( 1, 'maior', df ) )
print( fnc_calc_per_country( 1, 'grafico', df ) )

##### 2. Qual o nome do país que possui mais restaurantes registrados?

In [ ]:
print( fnc_calc_per_country( 2, 'maior', df ) )
print( fnc_calc_per_country( 2, 'grafico', df ) )

##### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [ ]:
print( fnc_calc_per_country( 3, 'maior', df ) )
print( fnc_calc_per_country( 3, 'grafico', df ) )

##### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
print( fnc_calc_per_country( 4, 'maior', df ) )
print( fnc_calc_per_country( 4, 'grafico', df ) )

##### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
print( fnc_calc_per_country( 5, 'maior', df ) )
print( fnc_calc_per_country( 5, 'grafico', df ) )

##### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
print( fnc_calc_per_country( 6, 'maior', df ) )
print( fnc_calc_per_country( 6, 'grafico', df ) )

##### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [ ]:
print( fnc_calc_per_country( 7, 'maior', df ) )
print( fnc_calc_per_country( 7, 'grafico', df ) )

##### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [ ]:
df_aux = df.loc[ :, [ 'votes', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( [ 'votes' ], ascending=False ).reset_index()

df_aux['country_name'][0]

##### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [ ]:
df_aux = df.loc[ :, [ 'aggregate_rating', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( [ 'aggregate_rating' ], ascending=False ).reset_index()

df_aux['country_name'][0]

##### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [ ]:
df_aux = df.loc[ :, [ 'aggregate_rating', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( [ 'aggregate_rating' ], ascending=True ).reset_index()

# country_name( df_aux['country_code'][0] )
df_aux['country_name'][0]

##### 11. Qual a média de preço de um prato para dois por país?

In [ ]:
df.loc[ :, [ 'average_cost_for_two', 'country_name' ] ].groupby( [ 'country_name' ] ).mean().sort_values( 'average_cost_for_two', ascending=False ).reset_index()

### Cidade

##### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [ ]:
df_aux = df.loc[ :, [ 'restaurant_id', 'city', 'country_name' ] ].groupby( [ 'city', 'country_name' ] ).count().sort_values( ['restaurant_id'], ascending = False ).reset_index().head(10)
df_aux.columns = [ 'Cidade', 'País', 'Quantidade de Restaurantes' ]
df_aux

In [ ]:
df_aux = df.loc[ :, [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( ['restaurant_id'], ascending = False ).reset_index().head(10)
df_aux['city'][0]

df_aux

##### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [ ]:
df.loc[ ( df[ 'aggregate_rating' ] > 4 ), [ 'restaurant', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant' ], ascending=False ).reset_index().head(1)
df_aux['city'][0]

##### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [ ]:
df_aux = df.loc[ ( df[ 'aggregate_rating' ] < 2.5 ), [ 'restaurant', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant' ], ascending=False ).reset_index().head(1)
df_aux['city'][0]

##### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [ ]:
df_aux = df.loc[ :, [ 'average_cost_for_two', 'city' ] ].groupby( [ 'city' ] ).mean().sort_values( [ 'average_cost_for_two' ], ascending=False ).reset_index().head(1)
df_aux['city'][0]

##### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
df_aux01 = df.loc[ :, ['country_name', 'city', 'cuisines']]
df_aux01 = df_aux01.groupby( ['country_name', 'city'] ).nunique().sort_values( ['country_name', 'cuisines'], ascending=False ).reset_index()
df_aux01

##### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
df_aux = df.loc[ ( df[ 'has_table_booking' ] == 1 ), [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant_id' ], ascending=False ).reset_index().head(1)
df_aux[ 'city' ][0]

##### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [ ]:
df_aux = df.loc[ ( df[ 'is_delivering_now' ] == 1 ), [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant_id' ], ascending=False ).reset_index().head(1)
df_aux[ 'city' ][0]

##### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
df_aux = df.loc[ ( df[ 'has_online_delivery' ] == 1 ), [ 'restaurant_id', 'city' ] ].groupby( [ 'city' ] ).count().sort_values( [ 'restaurant_id' ], ascending=False ).reset_index().head(1)
df_aux[ 'city' ][0]

### Restaurantes

##### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [ ]:
df_aux = df.loc[ :, [ 'votes', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] ).sum().sort_values( [ 'votes' ], ascending=False ).reset_index().head(1)
df_aux[ 'restaurant_name' ][0]

##### 2. Qual o nome do restaurante com a maior nota média?

In [ ]:
df_aux = df.loc[ :, [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] ).mean().sort_values( [ 'aggregate_rating' ], ascending=False ).reset_index().head(1)
df_aux[ 'restaurant_name' ][0]

##### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [ ]:
df_aux = df.loc[ :, [ 'average_cost_for_two', 'restaurant_name' ] ].sort_values( [ 'average_cost_for_two' ], ascending=False ).reset_index().head(1)
df_aux['restaurant_name'][0]

##### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?


In [ ]:
df_aux = (df.loc[ ( df[ 'cuisines' ] == 'Brazilian' ), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                  .mean()
                                                                                                  .sort_values( [ 'aggregate_rating' ] )
                                                                                                  .reset_index()
                                                                                                  .head(1) )
df_aux[ 'restaurant_name' ][0]

##### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Brazilian' ) & (( df[ 'country_name' ] == 'Brazil' )), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                                                            .mean()
                                                                                                                                            .sort_values( [ 'aggregate_rating' ], ascending=False )
                                                                                                                                            .reset_index()
                                                                                                                                            .head(1) )
df_aux[ 'restaurant_name' ][0]

##### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?


In [ ]:
df_pedido_online = df.loc[ ( df[ 'has_online_delivery' ] == 1 ), [ 'restaurant_id', 'restaurant_name' ] ].sort_values( [ 'restaurant_id' ] ).reset_index()
print( df_pedido_online )

df_maior_media = ( df.loc[ :, [ 'votes', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                              .mean()
                                                              .sort_values( [ 'votes' ], ascending=False )
                                                              .reset_index() )
print( df_maior_media )

##### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?


In [ ]:
df_reserva = df.loc[ ( df[ 'has_table_booking' ] == 1 ), [ 'restaurant_id', 'restaurant_name' ] ].sort_values( [ 'restaurant_id' ] ).reset_index()
print( df_reserva )

df_maior_media = ( df.loc[ :, [ 'average_cost_for_two', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                             .mean()
                                                                             .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                                             .reset_index() )
print( df_maior_media )

##### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americana (BBQ)?

In [ ]:
df_japa_eua = ( df.loc[ ( df[ 'country_name' ] == 'United States of America' ) & ( df[ 'cuisines' ] == 'Japanese' ), [ 'average_cost_for_two', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                                                                                    .mean()
                                                                                                                                                                    .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                                                                                                                                    .reset_index()
                                                                                                                                                                    .head(10) )
print( df_japa_eua )

df_cost_bbq = ( df.loc[ ( df[ 'cuisines' ] == 'BBQ' ), [ 'average_cost_for_two', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                                      .mean()
                                                                                                      .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                                                                      .reset_index()
                                                                                                      .head(10) )
print( df_cost_bbq )

### Tipos de Culinária

##### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Italian' ), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                        .mean()
                                                                                        .sort_values( [ 'aggregate_rating' ], ascending=False )
                                                                                        .reset_index()
                                                                                        .head(1) )
df_aux[ 'restaurant_name' ][0]

In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Italian' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                    .mean()
                                                                                                                    .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending=[ False, True ] )
                                                                                                                    .reset_index()
                                                                                                                    .head(15) )
df_aux

##### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Italian' ), [ 'aggregate_rating', 'restaurant_name' ] ].groupby( [ 'restaurant_name' ] )
                                                                                        .mean()
                                                                                        .sort_values( [ 'aggregate_rating' ] )
                                                                                        .reset_index()
                                                                                        .head(1) )
df_aux[ 'restaurant_name' ][0]

##### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'American' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_id', 'restaurant_name' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'American' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Arabian' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                  .mean()
                                                                                                                  .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                  .reset_index()
                                                                                                                  .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Arabian' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                  .mean()
                                                                                                                  .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                  .reset_index()
                                                                                                                  .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Japanese' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Japanese' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Home-made' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = [ False, True ] )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
df_aux = ( df.loc[ ( df[ 'cuisines' ] == 'Home-made' ), [ 'aggregate_rating', 'restaurant_name', 'restaurant_id' ] ].groupby( [ 'restaurant_name', 'restaurant_id' ] )
                                                                                                                   .mean()
                                                                                                                   .sort_values( [ 'aggregate_rating', 'restaurant_id' ], ascending = True )
                                                                                                                   .reset_index()
                                                                                                                   .head(50) )
df_aux[ 'restaurant_name' ][0]

##### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?


In [ ]:
df_aux = ( df.loc[ :, [ 'average_cost_for_two', 'cuisines' ] ].groupby( [ 'cuisines' ] )
                                                              .mean()
                                                              .sort_values( [ 'average_cost_for_two' ], ascending=False )
                                                              .reset_index() )

df_aux[ 'cuisines' ][0]

##### 12. Qual o tipo de culinária que possui a maior nota média?


In [ ]:
df_aux = ( df.loc[ :, [ 'aggregate_rating', 'cuisines' ] ].groupby( [ 'cuisines' ] )
                                                          .mean()
                                                          .sort_values( [ 'aggregate_rating' ], ascending=False )
                                                          .reset_index() )

df_aux[ 'cuisines' ][0]

##### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [ ]:
df_aux = ( df.loc[ ( df[ 'has_online_delivery' ] == 1 ) & ( df[ 'is_delivering_now' ] == 1 ), [ 'restaurant_id', 'cuisines' ] ].groupby( [ 'cuisines' ] )
                                                                                                                               .count()
                                                                                                                               .sort_values( [ 'restaurant_id' ], ascending=False )
                                                                                                                               .reset_index() )

df_aux[ 'cuisines' ][0]